O objetivo deste arquivo é decidir o nível de lags a ser aplicado ao LASSO

In [8]:
import utils
import epftools as epf
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError, MeanSquaredError, MeanAbsoluteError
from sklearn.linear_model import LassoCV
from sktime.forecasting.compose import make_reduction
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [9]:
df = utils.read_and_change_index("dados/lags_transformados.csv")
df.head()

,ipca,ipca_15,selic,m1,m2,m3,m4,ibovespa,cambio,cambio_real,...,crb_metal_lag1,crb_metal_lag2,crb_metal_lag3,crb_metal_lag4,crb_metal_lag5,crb_metal_lag6,crb_metal_lag7,crb_metal_lag8,crb_metal_lag9,crb_metal_lag10
month,,,,,,,,,,,,,,,,,,,,,
2004-01,0.76,0.68,-0.548387,-11559013.0,-22319905.0,-18028683.0,-17053471.0,-384.95,-0.0735,-0.363,...,27.42,2.99,17.20,15.18,-2.05,9.59,-0.18,2.52,1.59,-3.76
2004-02,0.61,0.90,0.000000,1650795.0,11117714.0,1275340.0,-2131936.0,-96.42,0.0785,1.699,...,15.10,27.42,2.99,17.20,15.18,-2.05,9.59,-0.18,2.52,1.59
2004-03,0.47,0.40,-0.112903,-1575433.0,-1282702.0,300227.0,-4199993.0,387.24,-0.0248,-2.777,...,24.27,15.10,27.42,2.99,17.20,15.18,-2.05,9.59,-0.18,2.52
2004-04,0.37,0.21,-0.270430,539382.0,-378329.0,-4736024.0,-2778970.0,-2535.03,0.0005,-1.562,...,10.95,24.27,15.10,27.42,2.99,17.20,15.18,-2.05,9.59,-0.18
2004-05,0.51,0.54,-0.116667,1252881.0,13051576.0,26890443.0,11143701.0,-62.56,0.1944,4.213,...,-11.13,10.95,24.27,15.10,27.42,2.99,17.20,15.18,-2.05,9.59


In [10]:
# Normaliza dados
scaler = StandardScaler()
normalized_array = scaler.fit_transform(df)
normalized_df = pd.DataFrame(normalized_array, columns=df.columns, index=df.index)
normalized_df.head()

,ipca,ipca_15,selic,m1,m2,m3,m4,ibovespa,cambio,cambio_real,...,crb_metal_lag1,crb_metal_lag2,crb_metal_lag3,crb_metal_lag4,crb_metal_lag5,crb_metal_lag6,crb_metal_lag7,crb_metal_lag8,crb_metal_lag9,crb_metal_lag10
month,,,,,,,,,,,,,,,,,,,,,
2004-01,0.888700,0.646061,-1.439646,-0.913960,-0.480973,-0.285827,-0.341305,-0.170365,-0.655551,-0.097193,...,0.565892,-0.006510,0.326224,0.278103,-0.131454,0.138006,-0.089679,-0.031767,-0.043630,-0.168206
2004-02,0.435360,1.312838,0.045303,-0.030459,0.232655,0.016213,-0.052490,-0.107584,0.559792,0.825071,...,0.278870,0.563361,-0.005414,0.325239,0.273100,-0.135670,0.140102,-0.095505,-0.021386,-0.040130
2004-03,0.012242,-0.202565,-0.260422,-0.246236,-0.031996,0.000956,-0.092518,-0.002344,-0.266162,-1.176894,...,0.492506,0.275976,0.564743,-0.006348,0.320529,0.269436,-0.133660,0.135132,-0.085966,-0.017866
2004-04,-0.289985,-0.778419,-0.686981,-0.104793,-0.012695,-0.077844,-0.065013,-0.638203,-0.063871,-0.633465,...,0.182187,0.489882,0.277214,0.563720,-0.013117,0.316929,0.271573,-0.139650,0.147717,-0.082503
2004-05,0.133133,0.221748,-0.270613,-0.057072,0.273927,0.416999,0.204468,-0.100216,1.486492,1.949499,...,-0.332217,0.179171,0.491227,0.276236,0.560490,-0.017171,0.319082,0.267093,-0.130694,0.151387


In [11]:
# Cria diferentes DataFrames, cada um com uma quantidade diferente de lags
# Variam de 0 a 10 lags
dfs = []
for i in range(1, 12):
    new_df = normalized_df.copy()
    for col in normalized_df:
        if "lag" in col:
            _, lag_n = col.split("lag")
            if int(lag_n) >= i:
                new_df = new_df.drop(columns=col)
    dfs.append(new_df)

In [12]:
# Só pra checar se funcionou como o esperado
dfs[5].head()

,ipca,ipca_15,selic,m1,m2,m3,m4,ibovespa,cambio,cambio_real,...,crb_food_lag1,crb_food_lag2,crb_food_lag3,crb_food_lag4,crb_food_lag5,crb_metal_lag1,crb_metal_lag2,crb_metal_lag3,crb_metal_lag4,crb_metal_lag5
month,,,,,,,,,,,,,,,,,,,,,
2004-01,0.888700,0.646061,-1.439646,-0.913960,-0.480973,-0.285827,-0.341305,-0.170365,-0.655551,-0.097193,...,-0.490951,0.062602,0.149444,0.782744,0.277480,0.565892,-0.006510,0.326224,0.278103,-0.131454
2004-02,0.435360,1.312838,0.045303,-0.030459,0.232655,0.016213,-0.052490,-0.107584,0.559792,0.825071,...,1.123738,-0.499147,0.068379,0.148767,0.782424,0.278870,0.563361,-0.005414,0.325239,0.273100
2004-03,0.012242,-0.202565,-0.260422,-0.246236,-0.031996,0.000956,-0.092518,-0.002344,-0.266162,-1.176894,...,0.869091,1.122177,-0.495834,0.067737,0.148489,0.492506,0.275976,0.564743,-0.006348,0.320529
2004-04,-0.289985,-0.778419,-0.686981,-0.104793,-0.012695,-0.077844,-0.065013,-0.638203,-0.063871,-0.633465,...,0.095507,0.866484,1.132602,-0.496231,0.067464,0.182187,0.489882,0.277214,0.563720,-0.013117
2004-05,0.133133,0.221748,-0.270613,-0.057072,0.273927,0.416999,0.204468,-0.100216,1.486492,1.949499,...,0.329576,0.089721,0.875788,1.131496,-0.496466,-0.332217,0.179171,0.491227,0.276236,0.560490


In [13]:
# Compara modelos com lags diferentes
results_dict = {}
y_preds = {}
for i, d in enumerate(dfs):
    regressor = LassoCV()
    forecaster = make_reduction(regressor)
    y_train, y_test, y_pred = utils.evaluate_pipeline(forecaster, d)

    # Desnormaliza previsões
    y_pred = y_pred * np.sqrt(scaler.var_[0]) + scaler.mean_[0]
    y_preds[f'lags_{i}'] = y_pred
    
    mae = MeanAbsoluteError()
    mse = MeanSquaredError()
    mape = MeanAbsolutePercentageError()

    test_array = y_test.to_numpy()
    pred_array = y_pred.to_numpy()
    rss = np.abs(test_array - pred_array).sum()

    results_dict[f'lags_{i}'] = [
        mae(y_test, y_pred),
        mse(y_test, y_pred),
        mape(y_test, y_pred),
        rss
    ]

C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.039e-03, tolerance: 8.957e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.159e-03, tolerance: 8.957e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\loca

In [14]:
res_df = pd.DataFrame(results_dict, index=['MAE', 'MSE', 'MAPE', 'RSS'])
res_df

,lags_0,lags_1,lags_2,lags_3,lags_4,lags_5,lags_6,lags_7,lags_8,lags_9,lags_10
MAE,1.082634,1.080391,1.077956,1.076677,1.078732,1.076796,1.074370,1.074427,1.073918,1.075073,1.075474
MSE,1.721584,1.719013,1.708652,1.697125,1.702057,1.696118,1.687882,1.691359,1.692704,1.696409,1.699194
MAPE,2.589979,2.565990,2.551127,2.517226,2.511224,2.549607,2.486409,2.478002,2.437405,2.443254,2.439851
RSS,85.528090,85.350853,85.158528,85.057468,85.219867,85.066905,84.875219,84.879706,84.839519,84.930803,84.962458


In [15]:
preds_df = pd.DataFrame(y_preds)
preds_df

,lags_0,lags_1,lags_2,lags_3,lags_4,lags_5,lags_6,lags_7,lags_8,lags_9,lags_10
month,,,,,,,,,,,
2017-03,0.413438,0.416986,0.401294,0.398109,0.403037,0.398599,0.405359,0.382455,0.390300,0.410898,0.418926
2017-04,0.425752,0.425728,0.427231,0.421577,0.420028,0.419920,0.387517,0.359491,0.352280,0.345234,0.345117
2017-05,0.123628,0.123937,0.104603,0.118974,0.124113,0.142377,0.105116,0.144774,0.139185,0.148358,0.148013
2017-06,0.198299,0.198146,0.219808,0.200968,0.201702,0.200551,0.203655,0.172472,0.169427,0.175226,0.175024
2017-07,0.042775,0.040753,0.042276,0.010362,0.010138,0.005710,0.017895,-0.015135,-0.041386,-0.050933,-0.051062
...,...,...,...,...,...,...,...,...,...,...,...
2023-05,0.599937,0.595117,0.528560,0.469295,0.466366,0.463121,0.463806,0.463116,0.465936,0.462734,0.462077
2023-06,0.326544,0.282730,0.208332,0.302105,0.302879,0.290211,0.294720,0.295991,0.295244,0.297489,0.297122
2023-07,-0.006965,0.011428,-0.084640,-0.146815,-0.134398,-0.106500,-0.081493,-0.084575,-0.079900,-0.059812,-0.060030


In [16]:
p_values = []
for col in preds_df:
    if col == "lags_0":
        continue
    p_values.append(epf.DM(y_test, preds_df['lags_0'], preds_df[col]))
p_values

[0.1522208893548389,
 0.17921139384162277,
 0.17651949460996197,
 0.27558583358706823,
 0.21246635773433808,
 0.15693101580245994,
 0.1770864024127239,
 0.1703724457374638,
 0.20850166258218716,
 0.22360912922926812]